In [1]:
import re
import pandas as pd
import sentencepiece as spm

from mecab import MeCab
from tokenizers import SentencePieceBPETokenizer

In [2]:
data_path = "../dataset"

train_df = pd.read_csv(f"{data_path}/cleaned_train.csv")
valid_df = pd.read_csv(f"{data_path}/cleaned_dev.csv")

print(train_df.shape)
print(train_df.head())
print()

print(valid_df.shape)
print(valid_df.head())

(12457, 4)
     fname                                           dialogue  \
0  train_0  #Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...   
1  train_1  #Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...   
2  train_2  #Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...   
3  train_3  #Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?\n#Person...   
4  train_4  #Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 ...   

                                             summary     topic  
0  스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...   건강검진 받기  
1  파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...        백신  
2  #Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...     열쇠 찾기  
3  #Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...  여자친구가 있다  
4  말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...        댄스  

(499, 4)
   fname                                           dialogue  \
0  dev_0  #Person1#: 안녕하세요, 오늘 하루 어떠셨어요? \n#Person2#: 요즘...   
1  dev_1  #Person1#: 헤이, 지미. 나중에 운동하러 가자.\n#Person2#: 그래...   
2  dev_2 

In [3]:
train_df['is_train'] = 1
valid_df['is_train'] = 0
total_df = pd.concat([train_df, valid_df], ignore_index=True)

In [4]:
## [#문자열#:]을 제거
def remove_speaker_tags(text):
    return re.sub(r'#\w+#:', '', text)

## 개행 문자 제거.
def remove_newlines(text):
    # 개행 문자를 문장부호 뒤에 공백과 함께 추가
    text = re.sub(r'([.?!])\n', r'\1 ', text)

    # 개행 문자가 남아있는 경우 제거
    return text.replace('\n', ' ')

In [5]:
## Sample
cleaned_dialogue_sample = remove_speaker_tags(total_df['dialogue'].iloc[0])
cleaned_dialogue_sample = remove_newlines(cleaned_dialogue_sample)

mecab = MeCab()
morphs = mecab.morphs(cleaned_dialogue_sample)
print(morphs)

['안녕', '하', '세요', ',', '스미스', '씨', '.', '저', '는', '호킨스', '의사', '입니다', '.', '오늘', '왜', '오', '셨', '나요', '?', '건강', '검진', '을', '받', '는', '것', '이', '좋', '을', '것', '같', '아서요', '.', '그렇', '군요', ',', '당신', '은', '5', '년', '동안', '건강', '검진', '을', '받', '지', '않', '았', '습니다', '.', '매년', '받', '아야', '합니다', '.', '알', '고', '있', '습니다', '.', '하지만', '아무', '문제', '가', '없', '다면', '왜', '의사', '를', '만나', '러', '가', '야', '하', '나요', '?', '심각', '한', '질병', '을', '피하', '는', '가장', '좋', '은', '방법', '은', '이', '를', '조기', '에', '발견', '하', '는', '것', '입니다', '.', '그러', '니', '당신', '의', '건강', '을', '위해', '최소한', '매년', '한', '번', '은', '오', '세요', '.', '알', '겠', '습니다', '.', '여기', '보', '세요', '.', '당신', '의', '눈', '과', '귀', '는', '괜찮', '아', '보', '입니다', '.', '깊', '게', '숨', '을', '들이쉬', '세요', '.', '스미스', '씨', ',', '담배', '피우', '시', '나요', '?', '네', '.', '당신', '도', '알', '다시피', ',', '담배', '는', '폐암', '과', '심장', '병', '의', '주요', '원인', '입니다', '.', '정말로', '끊', '으셔야', '합니다', '.', '수백', '번', '시도', '했', '지만', ',', '습관', '을', '버리', '는', '것', '이', '어렵', '습

In [8]:
mecab = MeCab()

# Dialogue와 Summary 형태소 분석
total_df['cleaned_dialogue'] = total_df['dialogue'].apply(lambda x: remove_newlines(remove_speaker_tags(x)))
total_df['cleaned_summary'] = total_df['summary'].apply(lambda x: remove_newlines(remove_speaker_tags(x)))

total_df['dialogue_morphs'] = total_df['cleaned_dialogue'].apply(lambda x: mecab.morphs(x))
total_df['summary_morphs'] = total_df['cleaned_summary'].apply(lambda x: mecab.morphs(x))

total_df.head()

,fname,dialogue,summary,topic,is_train,cleaned_dialogue,cleaned_summary,dialogue_morphs,summary_morphs
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기,1,"안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나요? 건강검진을 ...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...","[안녕, 하, 세요, ,, 스미스, 씨, ., 저, 는, 호킨스, 의사, 입니다, ...","[스미스, 씨, 가, 건강, 검진, 을, 받, 고, 있, 고, ,, 호킨스, 의사,..."
1,train_1,"#Person1#: 안녕하세요, 파커 부인, 어떻게 지내셨나요?\n#Person2#...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,백신,1,"안녕하세요, 파커 부인, 어떻게 지내셨나요? 안녕하세요, 피터스 박사님. 잘 지...",파커 부인이 리키를 데리고 백신 접종을 하러 갔다. 피터스 박사는 기록을 확인한 후...,"[안녕, 하, 세요, ,, 파커, 부인, ,, 어떻게, 지내, 셨, 나요, ?, 안...","[파커, 부인, 이, 리키, 를, 데리, 고, 백신, 접종, 을, 하, 러, 갔, ..."
2,train_2,"#Person1#: 실례합니다, 열쇠 한 묶음 보셨나요?\n#Person2#: 어떤...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...",열쇠 찾기,1,"실례합니다, 열쇠 한 묶음 보셨나요? 어떤 종류의 열쇠인가요? 5개의 열쇠와 ...","#Person1#은 열쇠 한 묶음을 찾고 있고, 그것을 찾기 위해 #Person2#...","[실례, 합니다, ,, 열쇠, 한, 묶음, 보, 셨, 나요, ?, 어떤, 종류, 의...","[#, Person, 1, #, 은, 열쇠, 한, 묶음, 을, 찾, 고, 있, 고,..."
3,train_3,#Person1#: 왜 너는 여자친구가 있다는 걸 말해주지 않았어?\n#Person...,#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...,여자친구가 있다,1,"왜 너는 여자친구가 있다는 걸 말해주지 않았어? 미안해, 너가 이미 알고 있다고...",#Person1#은 #Person2#가 여자친구가 있고 그녀와 결혼할 것이라는 사실...,"[왜, 너, 는, 여자, 친구, 가, 있, 다는, 걸, 말, 해, 주, 지, 않, ...","[#, Person, 1, #, 은, #, Person, 2, #, 가, 여자, 친..."
4,train_4,"#Person1#: 안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 ...",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...,댄스,1,"안녕, 숙녀분들! 오늘 밤 당신들은 정말 멋져 보여. 이 춤을 나와 함께 해줄래?...",말릭이 니키에게 춤을 요청한다. 말릭이 발을 밟는 것을 신경 쓰지 않는다면 니키는 ...,"[안녕, ,, 숙녀, 분, 들, !, 오늘, 밤, 당신, 들, 은, 정말, 멋져, ...","[말릭, 이, 니키, 에게, 춤, 을, 요청, 한다, ., 말, 릭, 이, 발, 을..."


In [11]:
corpus = total_df['dialogue_morphs'].tolist() + total_df['summary_morphs'].tolist()
corpus = [' '.join(morphs) for morphs in corpus]

with open('./spm_corpus.txt', 'w') as f:
    for line in corpus:
        f.write(f"{line}\n")

In [13]:
spm.SentencePieceTrainer.train(
    input='./spm_corpus.txt', 
    model_prefix='./spm_unigram', 
    vocab_size=8784,  # 원하는 vocab 크기로 조정 가능
    model_type='unigram'
)

sp = spm.SentencePieceProcessor(model_file='./spm_unigram.model')
sample_text = ' '.join(total_df['dialogue_morphs'].iloc[0])
print(f"Original Text: {sample_text}")
print(f"Tokenized: {sp.encode_as_pieces(sample_text)}")

Original Text: 안녕 하 세요 , 스미스 씨 . 저 는 호킨스 의사 입니다 . 오늘 왜 오 셨 나요 ? 건강 검진 을 받 는 것 이 좋 을 것 같 아서요 . 그렇 군요 , 당신 은 5 년 동안 건강 검진 을 받 지 않 았 습니다 . 매년 받 아야 합니다 . 알 고 있 습니다 . 하지만 아무 문제 가 없 다면 왜 의사 를 만나 러 가 야 하 나요 ? 심각 한 질병 을 피하 는 가장 좋 은 방법 은 이 를 조기 에 발견 하 는 것 입니다 . 그러 니 당신 의 건강 을 위해 최소한 매년 한 번 은 오 세요 . 알 겠 습니다 . 여기 보 세요 . 당신 의 눈 과 귀 는 괜찮 아 보 입니다 . 깊 게 숨 을 들이쉬 세요 . 스미스 씨 , 담배 피우 시 나요 ? 네 . 당신 도 알 다시피 , 담배 는 폐암 과 심장 병 의 주요 원인 입니다 . 정말로 끊 으셔야 합니다 . 수백 번 시도 했 지만 , 습관 을 버리 는 것 이 어렵 습니다 . 우리 는 도움 이 될 수 있 는 수업 과 약물 들 을 제공 하 고 있 습니다 . 나가 기 전 에 더 많 은 정보 를 드리 겠 습니다 . 알 겠 습니다 , 감사 합니다 , 의사 선생 님 .
Tokenized: ['▁안녕', '▁하', '▁세요', '▁,', '▁스미', '스', '▁씨', '▁.', '▁저', '▁는', '▁호', '킨', '스', '▁의사', '▁입니다', '▁.', '▁오늘', '▁왜', '▁오', '▁셨', '▁나', '요', '▁?', '▁건강', '▁검', '진', '▁을', '▁받', '▁는', '▁', '것', '▁이', '▁좋', '▁을', '▁', '것', '▁같', '▁아서', '요', '▁.', '▁그렇', '▁군요', '▁,', '▁당', '신', '▁은', '▁5', '▁', '년', '▁동', '안', '▁건강', '▁검', '진', '▁을', '▁받', '▁지', '▁', '않', '▁았', '▁습니다', '▁.', '▁매', '년', '▁받', '▁아야', '▁합니다', '▁.', '

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: ./spm_corpus.txt
  input_format: 
  model_prefix: ./spm_unigram
  model_type: UNIGRAM
  vocab_size: 8784
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 

In [19]:
from transformers import AutoTokenizer
from transformers import PreTrainedTokenizer

class SentencePieceTokenizer(PreTrainedTokenizer):
    def __init__(self, sp_model_path):
        super().__init__()
        self.sp = spm.SentencePieceProcessor()
        self.sp.load(sp_model_path)

    def _tokenize(self, text):
        return self.sp.encode_as_pieces(text)

    def _convert_token_to_id(self, token):
        return self.sp.piece_to_id(token)

    def _convert_id_to_token(self, index):
        return self.sp.id_to_piece(index)

    def convert_tokens_to_string(self, tokens):
        return ''.join(tokens).replace('▁', ' ').strip()
    
tokenizer = SentencePieceTokenizer('./spm_unigram.model')

/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


NotImplementedError: 

In [20]:
from tokenizers import SentencePieceBPETokenizer

tokenizer = SentencePieceBPETokenizer(
    "./spm_unigram.model",
    unk_token="<unk>"
)

tokenizer.save("./tokenizer.json")